In [65]:
import pickle

import pandas as pd

with open('doc_returns.pkl', 'rb') as f:
    doc_returns = pickle.load(f)

In [66]:
with open('../sentiment_analysis/topic_sentiment_analysis.pkl', 'rb') as f:
    topic_sentiment_analysis = pickle.load(f)
topic_sentiment_analysis.dropna(inplace=True, how='all')
topic_sentiment_analysis.head()

,123779,123730,123731,123678,123676,123677,123615,123483,123280,123281,...,66920,84387,45581,83759,69773,100780,43397,86265,100779,98672
topic,,,,,,,,,,,,,,,,,,,,,
credit,NaN,0.178673,-0.001035,NaN,NaN,NaN,0.283389,0.033357,0.498359,0.289802,...,-0.498845,NaN,NaN,0.249558,-0.026119,NaN,NaN,NaN,NaN,NaN
dealing terms,NaN,-0.005857,0.214961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000111,0.000326,NaN,-0.000041,0.001717,0.000390,0.009697,0.001717,-0.164886
fees,0.097609,0.166250,0.000069,NaN,0.000074,NaN,0.000029,0.001532,0.000131,0.000115,...,0.495658,0.000237,0.038349,NaN,0.046134,0.000257,0.000098,NaN,0.000257,0.996133
investment approach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.934497,NaN,NaN,NaN,0.338085,NaN,NaN,NaN,NaN,NaN
market,-0.142190,-0.001087,0.000067,NaN,0.000006,NaN,0.047601,-0.230250,0.123491,0.214307,...,0.201671,NaN,-0.048678,-0.123748,0.041713,0.000032,-0.497505,NaN,0.000032,0.283777


In [67]:
results = pd.merge(doc_returns, topic_sentiment_analysis.T, left_index=True, right_index=True, how='inner')
factor_cols = [col for col in results.columns if col != 'avg_return']
results.head()

,avg_return,credit,dealing terms,fees,investment approach,market,organization,performance,portfolio,stress,team
123779,-0.0009,NaN,NaN,0.097609,NaN,-0.142190,NaN,-0.017482,0.273391,NaN,0.000031
123731,2.6349,-0.001035,0.214961,0.000069,NaN,0.000067,NaN,0.080113,0.246493,NaN,0.000324
123678,0.1000,NaN,NaN,NaN,NaN,NaN,-0.000028,NaN,NaN,NaN,NaN
123676,0.3317,NaN,NaN,0.000074,NaN,0.000006,NaN,-0.083941,-0.103765,NaN,0.002166
123677,1.5650,NaN,NaN,NaN,NaN,NaN,-0.000028,NaN,NaN,NaN,NaN


In [68]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
results_cp = results.copy()

results_cp[factor_cols] = scaler.fit_transform(results[factor_cols])
results_cp

,avg_return,credit,dealing terms,fees,investment approach,market,organization,performance,portfolio,stress,team
123779,-0.0009,NaN,NaN,0.098464,NaN,-0.141847,NaN,-0.017069,0.274342,NaN,0.000484
123731,2.6349,-0.000540,0.215199,0.000614,NaN,0.000708,NaN,0.080721,0.247390,NaN,0.000778
123678,0.1000,NaN,NaN,NaN,NaN,NaN,0.000497,NaN,NaN,NaN,NaN
123676,0.3317,NaN,NaN,0.000619,NaN,0.000647,NaN,-0.083660,-0.103564,NaN,0.002623
123677,1.5650,NaN,NaN,NaN,NaN,NaN,0.000497,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
89742,-0.0700,0.000087,-0.000038,-0.021486,NaN,-0.091618,0.000940,-0.049858,-0.032302,NaN,0.000528
101535,1.7600,NaN,NaN,NaN,NaN,-0.323449,NaN,0.195782,0.000438,NaN,0.000487
101536,3.4600,NaN,NaN,NaN,NaN,-0.323449,NaN,0.195782,0.000438,NaN,0.000487
101539,-5.8500,NaN,NaN,NaN,NaN,-0.323449,NaN,0.195782,0.000438,NaN,0.000487


In [69]:
# Old approach
from scipy.stats import pearsonr

# results.corr()
corr = pd.DataFrame(columns=['p', 'r'])
for col in results.columns:
    if col != 'avg_return':
        factor = results[results[col] > 0]
        print(f'Factor: {col}, N: {len(factor)}')
        p,r = pearsonr(factor['avg_return'], factor[col])
        corr.loc[col] = [p, r]
corr

Factor: credit, N: 186
Factor: dealing terms, N: 196
Factor: fees, N: 299
Factor: investment approach, N: 84
Factor: market, N: 274
Factor: organization, N: 158
Factor: performance, N: 304
Factor: portfolio, N: 347
Factor: stress, N: 35
Factor: team, N: 328


,p,r
credit,-0.034087,0.644166
dealing terms,0.058710,0.413710
fees,-0.070377,0.224997
investment approach,-0.029637,0.788993
market,0.047856,0.430121
organization,0.092813,0.246094
performance,0.023400,0.684472
portfolio,0.034314,0.524068
stress,0.230613,0.182589
team,0.007175,0.897008


In [70]:
x = results_cp[factor_cols].fillna(0)
y = results_cp['avg_return']

In [71]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(x, y)
coefficients = pd.DataFrame(reg.coef_, index=results_cp.columns[1:], columns=['coefficient'])
coefficients

,coefficient
credit,-0.677262
dealing terms,0.202756
fees,-1.408873
investment approach,-0.137089
market,0.195091
organization,0.633069
performance,0.486382
portfolio,0.401551
stress,0.622487
team,0.571358


In [72]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
lasso = Lasso(alpha=0.01)
lasso.fit(X_train, y_train)
coefficients = pd.DataFrame(lasso.coef_, index=results_cp.columns[1:], columns=['coefficient'])
coefficients

,coefficient
credit,-0.427301
dealing terms,-0.000000
fees,-1.201518
investment approach,-0.563320
market,0.000000
organization,0.720108
performance,0.260454
portfolio,0.669705
stress,0.758092
team,0.437804
